# Zipline Benchmarks

In [1]:
import pandas as pd
print(pd.__version__)
start, end = pd.Timestamp('2013-01-03', tz='UTC'), pd.Timestamp('2018-01-03', tz='UTC')

from zipline.pipeline import Pipeline
from zipline.utils.calendars import get_calendar
from zipline.data import bundles
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine

0.22.0


In [2]:
bundle_data = bundles.load('quandl')
zp_loader = USEquityPricingLoader(
            bundle_data.equity_daily_bar_reader, 
            bundle_data.adjustment_reader)

trading_calendar = get_calendar('NYSE') 

zpl_engine = SimplePipelineEngine(
    get_loader=lambda x: zp_loader,
    calendar=trading_calendar.all_sessions,
    asset_finder=bundle_data.asset_finder)

In [3]:
from zipline.pipeline.factors import SimpleMovingAverage, ExponentialWeightedMovingAverage
ma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=100)
pipeline = Pipeline()
pipeline.add( ma, 'ma') 
%time dfzp=zpl_engine.run_pipeline(pipeline, start, end)
dfzp.head(5).append(dfzp.tail(5))

Wall time: 3.54 s


ma
2013-01-03 00:00:00+00:00 Equity(0 [A])        38.110535
                          Equity(2 [AAL])      11.776900
                          Equity(3 [AAMC])     67.738462
                          Equity(4 [AAN])      28.779516
                          Equity(6 [AAON])     19.606132
2018-01-03 00:00:00+00:00 Equity(3190 [ZIXI])   4.824444
                          Equity(3194 [ZNGA])   3.831677
                          Equity(3195 [ZOES])  13.582525
                          Equity(3197 [ZTS])   66.604667
                          Equity(3198 [ZUMZ])  17.432071

In [4]:
%timeit zpl_engine.run_pipeline(pipeline, start, end)
%timeit -n 3 -r 10 zpl_engine.run_pipeline(pipeline, start, end)

3.21 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.98 s ± 14.4 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [19]:
pipeline = Pipeline()
ema = ExponentialWeightedMovingAverage.from_span(inputs=[USEquityPricing.close], span=50, window_length=200)
pipeline.add( ema, 'ema50')
%time dfzp=zpl_engine.run_pipeline(pipeline, start, end)
dfzp.head(5).append(dfzp.tail(5))

Wall time: 8.94 s


ema50
2013-01-03 00:00:00+00:00 Equity(0 [A])        38.895058
                          Equity(2 [AAL])      12.636658
                          Equity(3 [AAMC])           NaN
                          Equity(4 [AAN])      28.475159
                          Equity(6 [AAON])     20.192312
2018-01-03 00:00:00+00:00 Equity(3190 [ZIXI])        NaN
                          Equity(3194 [ZNGA])        NaN
                          Equity(3195 [ZOES])        NaN
                          Equity(3197 [ZTS])         NaN
                          Equity(3198 [ZUMZ])        NaN

In [20]:
%timeit zpl_engine.run_pipeline(pipeline, start, end)
%timeit -n 3 -r 10 zpl_engine.run_pipeline(pipeline, start, end)

7.59 s ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.6 s ± 15.4 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [5]:
from zipline.pipeline.factors import MovingAverageConvergenceDivergenceSignal
pipeline = Pipeline()
macd = MovingAverageConvergenceDivergenceSignal()
pipeline.add( macd, 'macd')
%time dfzp=zpl_engine.run_pipeline(pipeline, start, end)
dfzp.head(5).append(dfzp.tail(5))

Wall time: 9.03 s


macd
2013-01-03 00:00:00+00:00 Equity(0 [A])        0.848136
                          Equity(2 [AAL])      0.192983
                          Equity(3 [AAMC])          NaN
                          Equity(4 [AAN])     -0.252407
                          Equity(6 [AAON])    -0.157562
2018-01-03 00:00:00+00:00 Equity(3190 [ZIXI])  0.014607
                          Equity(3194 [ZNGA])  0.011381
                          Equity(3195 [ZOES])  0.516721
                          Equity(3197 [ZTS])   0.245217
                          Equity(3198 [ZUMZ])  0.228844

In [9]:
%timeit zpl_engine.run_pipeline(pipeline, start, end)
%timeit -n 3 -r 10 zpl_engine.run_pipeline(pipeline, start, end)

8.53 s ± 31.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.6 s ± 78.2 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [14]:
from zipline.pipeline.factors import RSI, FastStochasticOscillator
pipeline = Pipeline()
f = MovingAverageConvergenceDivergenceSignal() + RSI() + FastStochasticOscillator()
pipeline.add( f.rank().zscore(), 'f')
%time dfzp=zpl_engine.run_pipeline(pipeline, start, end)
dfzp.head(5).append(dfzp.tail(5))

Wall time: 16.6 s


f
2013-01-03 00:00:00+00:00 Equity(0 [A])        0.186959
                          Equity(2 [AAL])      0.927651
                          Equity(3 [AAMC])          NaN
                          Equity(4 [AAN])      0.103602
                          Equity(6 [AAON])    -0.502527
2018-01-03 00:00:00+00:00 Equity(3190 [ZIXI])  0.082782
                          Equity(3194 [ZNGA]) -1.163899
                          Equity(3195 [ZOES]) -0.736546
                          Equity(3197 [ZTS])  -1.023806
                          Equity(3198 [ZUMZ]) -1.103050

In [11]:
%timeit zpl_engine.run_pipeline(pipeline, start, end)
%timeit -n 3 -r 10 zpl_engine.run_pipeline(pipeline, start, end)

14.4 s ± 526 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
14.3 s ± 277 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)
